
# Hotel bookings

In this labo, we will exercice cleaning data on a dataset called 'Hotel Bookings'. The Hotel Booking demand dataset contains booking information for a city hotel and a resort hotel. It includes information such as booking time, length of stay, number of adults, children/babies, number of available parking spaces, among other things.
The City Hotel from this dataset asked us to predict which clients will cancel. We will do the prediction in a later labo.

Some of the code is already given. If you see any '...', then you need to add the right code yourself. Towards the end of the labo, less and less of the code will be given in advance and you will have to completely write the code yourself. If you get stuck, take a look at the code in the demo.

## 0. Loading packages and dataset

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-darkgrid')   

In [ ]:
df = pd.read_csv('hotel_bookings.csv')

## 1. Data exploration

## a) The basics
First, you have to explore the data. Find out the following things:
1. Number of features
2. Number of observations
3. The different datatypes that can be found in the dataset + the number of features per datatype
4. The number of features that have missing data
5. The amount of missing data for the features that have missing data


1. and 2: number of features and observations

In [ ]:
df.shape

3. Datatypes

In [ ]:
df.info()

4. and 5. Missing data

In [ ]:
df.isnull().sum()[df.isnull().sum() != 0]

or in percentages:

In [ ]:
(119390- df.isnull().sum()[df.isnull().sum() != 0])/119390*100

## b) Look at the data
Look at the first 5 rows of the data.You won't be able to see all features at once.
1. Do the columns and values make sense?
2. Look at the features.
    + Are there any features that are categorical, but not yet declared an object? If so, tell python.
3. There are dates in this dataset. We didn't see yet how to handle dates. 
    + If the dates are already split up into different features (day-month-year), these can be left as is (except for maybe making the months categorical)
    + If the full date is given in one feature
        + you have to first convert the feature type to a data, using pd.to_datetime(feature)
        + Then you have to extract the day, week, month  and year
            + feature.dt.year
            + feature.dt.month
            + feature.dt.week
            + feature.dt.day


1. Look at the data - does it make sense?

First take a look at feature 0 to 16

In [ ]:
df.iloc[:,0:16].head()

And then to feature 16 to 32

In [ ]:
df.iloc[:,16:32].head()

2. Convert categorical

In [ ]:

df['agent'] = df['agent'].astype('object')
df['company'] = df['company'].astype('object')


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

3. Deal with features that are dates.

In [ ]:
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

In [ ]:
df['reservation_status_date_year'] = df['reservation_status_date'].dt.year
df['reservation_status_date_month'] = df['reservation_status_date'].dt.month
df['reservation_status_date_week'] = df['reservation_status_date'].dt.week
df['reservation_status_date_day'] = df['reservation_status_date'].dt.day
df = df.drop(['reservation_status_date'],1)
df.info()

## c) Look at the descriptives
1. For which features do you suspect outliers?
2. Which of these outliers seem most suspicious? Which would you certainly check if you were able to?
3. For which years do you have an arrival date?
4. For which years do you have a reservation date?

First take a look at feature 0 to 18

In [ ]:
df.iloc[:,0:18].describe(include='all')

Then to feature 18 to 35

In [ ]:
df.iloc[:,18:35].describe()

## d) Plot the distributions
1. Make histograms for the numerical features
2. Make barplots for the categorical features: which features have sparse classes?

In [ ]:
df.hist( figsize = (12,14))

In [ ]:
import seaborn as sns

categorical = ['hotel','arrival_date_month','meal','country','market_segment','distribution_channel','reserved_room_type', 
'assigned_room_type','deposit_type','agent','company','customer_type','reservation_status']
for i in categorical: 
    sns.countplot(y=df[i])
    plt.show()


## e) Study the associations

1. Make crosstabs between the outcome 'is_canceled' and the categorical features. Use the option normalize='index' to get proportions, 
    1. Which customer_type cancels the most? 
    2. Which hotel has most cancellations?
    3. Which room type always cancels? How can you explain this?
    4. What do you notice for the feature 'reservation_status'. What will you do with this feature? Does this  have any consequences for other features?
2. Make a heat map for all continuous features    


In [ ]:
for i in categorical: 
    print(pd.crosstab(df[i],df['is_canceled'], normalize='index'))


In [ ]:
corrmat = df.corr()
f, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(corrmat, cmap='RdYlGn');

## 2. Data cleaning

In the data cleaning step, we will fix most of the issues that we noticed during the exploratory analysis. 

## a) Remove unwanted observations and features
1. Remove duplicated observations
2. Remove irrelevant observations (hint: for which hotel do you work?):
3. Remove irrelevant/leaky features (hint: what did you notice during the previous excercice?)


1. Remove duplicated observations

In [ ]:
df_no_dup = df.copy()
df_no_dup.drop_duplicates()   # Remove the duplicates
print(df_no_dup.shape)               # check the remaining number of observations

2. Remove irrelevant observations

In [ ]:
df_city = df_no_dup.copy()
df_city = df_no_dup[df_no_dup['hotel'] == 'City Hotel']
df_city.info()

3. Remove irrelevant features

In [ ]:
df_city = df_city.drop(['hotel','reservation_status','reservation_status_date_year', 'reservation_status_date_month',    # insert the features that you want to drop
                       'reservation_status_date_week', 'reservation_status_date_day'],1)
df_city.info()

## b) Fix structural errors

Unless you found any structural errors, you can skip this step.

## c) Deal with unwanted outliers
1. Do you need to remove any outliers?



## d) Handle missing data
1. Find out which features still have missing data now that we only have the observations of the city hotel.
2. Impute the missing values of the continuous feature(s).
3. Is it a good idea to flag the missigness here?
4. Appoint the missing values to an extra category 'Undefined'. You will have to define the features as categorical first.
    + Why 'Undefined' instead of the usual 'Missing?'
5. Check if all the missingness is gone    

1. Finding missingness

In [ ]:
df_city.isnull().sum()[df_city.isnull().sum() != 0]

2. continuous feature(s)


In [ ]:
df_imputed = df_city.copy()

# import Imputer 
from sklearn.impute import SimpleImputer

# Create an imputer object that looks for 'Nan' values, then replaces them with the mean value of the feature by columns (axis=0)
mean_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

# Train the imputor on the dataset
mean_imputer = mean_imputer.fit(np.array(df_city['children']).reshape(-1, 1) )

# Apply the imputer to the dataset (This imputer can also be used on future datasets)
df_imputed['children'] = mean_imputer.transform(np.array(df_city['children']).reshape(-1, 1) )

In [ ]:
df_imputed.head()

4. Categorical features

In [ ]:
categorical = ['agent','company', 'country']


for col in categorical:
    df_imputed[col] = df_imputed[col].astype('category')
    df_imputed[col] = df_imputed[col].cat.add_categories('Undefined')
    df_imputed[col] = df_imputed[col].fillna('Undefined')
    df_imputed[col] = df_imputed[col].astype('object')


5. Check

In [ ]:
df_imputed.info()

## 3. Feature engineering


## a) Create new features

1. Create a new feature 'length_stays', by combining the information in 'stays_in_weekend_nights' and 'stays_in_week_nights'
2. Create a new feature 'total_previous_bookings'. Which features will you combine for this?


In [ ]:
df_imputed['length_stays'] = df_imputed['stays_in_weekend_nights'] + df_imputed['stays_in_week_nights']
df_imputed['total_previous_bookings'] = df_imputed['previous_cancellations'] + df_imputed['previous_bookings_not_canceled']

## b) Combine sparse classes

We had eleven categorical features with sparse classes.
1. Remake the barplots for these features.
2. combine the sparse classes
    + meal: combine 'FB', and 'HB' into 'FB/HB'
    + country: combine all countries with less than 1% of the total observations into 'Other'
    + market_segment: Combine 'Aviation', 'Corporate', 'Complementary', and 'Undefined' into 'Other'  
    + distribution_channel: Combine 'Corporate', 'GDS' and 'Undefined' into 'Other' 
    + reserved_room_type: Combine 'B', 'E', 'F', 'G','C','P'  into 'Other
    + assigned_room_type: Combine 'B', 'E', 'F','G','C','P' and 'K' into 'Other
    + deposit_type: combine 'Non Refund', and 'Refundable' into 'Deposit'
    + agent: combine all agents with less than 1% of the total observations into 'Other'
    + company: combine all companies with less than 1% of the total observations into 'Other'
    + customer_type: combine 'Contract' and 'Group' into 'Other'
 3. Check the changes by remaking the plots   



1. New plots

In [ ]:
categorical = ['meal','country','market_segment','distribution_channel','reserved_room_type', 
'assigned_room_type','deposit_type','agent','company','customer_type']
for i in categorical:
    sns.countplot(y=df_imputed[i])
    plt.show()

2. Combining sparse classes

In [ ]:
df_sparse = df_imputed.copy()

# meal
df_sparse['meal'][df_sparse['meal'] == 'HB'] = 'HB/FB'
df_sparse['meal'][df_sparse['meal'] == 'FB'] = 'HB/FB'
 

In [ ]:
#country
threshold_percent = 1

series = pd.value_counts(df_sparse['country'])
mask = (series / series.sum() * 100).lt(threshold_percent)
df_sparse['country']= np.where(df_sparse['country'].isin(series[mask].index),'Other', df_sparse['country'])
df_sparse['country'].value_counts()

In [ ]:
# market_segment
df_sparse['market_segment'][df_sparse['market_segment'] == 'Complementary'] = 'Other'
df_sparse['market_segment'][df_sparse['market_segment'] == 'Aviation'] = 'Other'
df_sparse['market_segment'][df_sparse['market_segment'] == 'Undefined'] = 'Other'
df_sparse['market_segment'][df_sparse['market_segment'] == 'Corpoeate'] = 'Corpoeate'

In [ ]:
# distribution_channel
df_sparse['distribution_channel'][df_sparse['distribution_channel'] == 'Corporate'] = 'Other'
df_sparse['distribution_channel'][df_sparse['distribution_channel'] == 'GDS'] = 'Other'
df_sparse['distribution_channel'][df_sparse['distribution_channel'] == 'Undefined'] = 'Other'
    

In [ ]:
# reserved_room_type
df_sparse['reserved_room_type'][df_sparse['reserved_room_type'] == 'B'] = 'Other'
df_sparse['reserved_room_type'][df_sparse['reserved_room_type'] == 'E'] = 'Other'
df_sparse['reserved_room_type'][df_sparse['reserved_room_type'] == 'F'] = 'Other'

df_sparse['reserved_room_type'][df_sparse['reserved_room_type'] == 'G'] = 'Other'
df_sparse['reserved_room_type'][df_sparse['reserved_room_type'] == 'C'] = 'Other'
df_sparse['reserved_room_type'][df_sparse['reserved_room_type'] == 'P'] = 'Other'

# assigned_room_type
df_sparse['assigned_room_type'][df_sparse['assigned_room_type'] == 'B'] = 'Other'
df_sparse['assigned_room_type'][df_sparse['assigned_room_type'] == 'E'] = 'Other'
df_sparse['assigned_room_type'][df_sparse['assigned_room_type'] == 'F'] = 'Other'

df_sparse['assigned_room_type'][df_sparse['assigned_room_type'] == 'G'] = 'Other'
df_sparse['assigned_room_type'][df_sparse['assigned_room_type'] == 'C'] = 'Other'
df_sparse['assigned_room_type'][df_sparse['assigned_room_type'] == 'P'] = 'Other'
df_sparse['assigned_room_type'][df_sparse['assigned_room_type'] == 'K'] = 'Other'


In [ ]:
#deposit_type
df_sparse['deposit_type'][df_sparse['deposit_type'] == 'Non Refund'] = 'Deposit'
df_sparse['deposit_type'][df_sparse['deposit_type'] == 'Refundable'] = 'Deposit'

In [ ]:
#agent
threshold_percent = 1

series = pd.value_counts(df_sparse['agent'])
mask = (series / series.sum() * 100).lt(threshold_percent)
df_sparse['agent']= np.where(df_sparse['agent'].isin(series[mask].index),'Other', df_sparse['agent'])
df_sparse['agent'].value_counts()

In [ ]:
#company
threshold_percent = 1

series = pd.value_counts(df_sparse['company'])
mask = (series / series.sum() * 100).lt(threshold_percent)
df_sparse['company']= np.where(df_sparse['company'].isin(series[mask].index),'Other', df_sparse['company'])
df_sparse['company'].value_counts()

In [ ]:
#Customer type
threshold_percent = 1

series = pd.value_counts(df_sparse['customer_type'])
mask = (series / series.sum() * 100).lt(threshold_percent)

3. Check

In [ ]:
categorical = ['meal','country','market_segment','distribution_channel',
'reserved_room_type','assigned_room_type','deposit_type','agent','company','customer_type']
for i in categorical:
    sns.countplot(y=df_sparse[i])
    plt.show()

## c) Create dummy variables (categorical features)
1. Use one-hot-encoding for all categorical features.


In [ ]:
df_sparse = df_sparse.drop(['country','agent','market_segment'],1)


for col in df_sparse:
    if df_sparse[col].dtype ==  'object':
        dummies = pd.get_dummies(df_sparse[col], dummy_na=False, prefix=col)  #create dummies
        df_sparse = pd.concat([df_sparse, dummies],axis=1)                   # add dummies to dataset
        df_sparse.drop(columns=[col], inplace=True)                           # delete original feature

## d) Do log-transformation if necessary
1. Do log-transformations on the skewed features
    + Find out which features are numeric
    + But first, change the type of 'is_repeated_guest' to type 'category'
    + Calculate the skewness of all features
    + Keep the features with skewness larger than 0.75, get their index and take the log

In [ ]:
# Change this feature to ype 'category'
df_sparse['is_repeated_guest'] = df_sparse['is_repeated_guest'].astype('category')

# Select the features with type 'int64' and 'float64'
num_feat = df_sparse.select_dtypes(include=['int64', 'float64']).columns



In [ ]:
from scipy.stats import skew

# calculate the skewness
skewed = df_sparse[num_feat].apply(lambda x: skew(x.dropna().astype(float)))

# Only keep features where skewness is larger than 0.75
skewed = skewed[skewed > 0.75]

# Get the indexes of these features
skewed = skewed.index

# Take the log
df_sparse[skewed] = np.log1p(df_sparse[skewed])

In [ ]:
# Storing cleaned dataset to use in class 3
df_sparse_save = df_sparse.copy()
df_sparse_save = df_sparse_save.sample(n=10000)
df_sparse_save.to_csv('hotel_bookings_cleaned.csv', index=False)

## e) Standardize continuous features
1. Standardize the continuous features using 
    + StandardScaler
    + MinMaxSCaler
    + RobustScaler

In [ ]:
# standardisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df_sparse[num_feat])
df_sparse[num_feat] = scaler.transform(df_sparse[num_feat])


#MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = StandardScaler()
scaler.fit(df_sparse[num_feat])
df_sparse[num_feat] = scaler.transform(df_sparse[num_feat])


#Robustscaler
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaler.fit(df_sparse[num_feat])
df_sparse[num_feat] = scaler.transform(df_sparse[num_feat])



Ok, now you know all the basics on how to clean a dataset. Let's try it on the data from your project!